In [1]:
%load_ext autoreload
%autoreload 2

In [472]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
import time

from datetime import datetime
import locale
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')

sys.path.append('/Users/i.tolkachev/projects/fifa_esports_battle/')

In [ ]:
import re
import random
import pandas as pd

In [474]:
DOMAIN_NAME + 'su/results.htm'

'https://www.marathonbet.ru/su/results.htm'

In [490]:
from config import DOMAIN_NAME, SEARCH_STRING, DATA_FOLDER

RESULTS_CLASS_NAME = "item-label__text"

DATE_START = '01.04.2020'
DATE_END = '07.04.2020'

In [491]:
dt_start = datetime.strptime(DATE_START, '%d.%m.%Y')
dt_end = datetime.strptime(DATE_END, '%d.%m.%Y')
for dt in pd.date_range(dt_start, dt_end):
    file_dt = dt.strftime('%Y_%m_%d')
    dt = dt.strftime('%Y-%m-%d')
    fname = DATA_FOLDER + f'games_{file_dt}.csv'

    print(f'Saving to file {fname}')

Saving to file /Users/i.tolkachev/projects/fifa_esports_battle/data/games_2020_04_01.csv
Saving to file /Users/i.tolkachev/projects/fifa_esports_battle/data/games_2020_04_02.csv
Saving to file /Users/i.tolkachev/projects/fifa_esports_battle/data/games_2020_04_03.csv
Saving to file /Users/i.tolkachev/projects/fifa_esports_battle/data/games_2020_04_04.csv
Saving to file /Users/i.tolkachev/projects/fifa_esports_battle/data/games_2020_04_05.csv
Saving to file /Users/i.tolkachev/projects/fifa_esports_battle/data/games_2020_04_06.csv
Saving to file /Users/i.tolkachev/projects/fifa_esports_battle/data/games_2020_04_07.csv


In [446]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [475]:
driver = webdriver.Firefox()
driver.get(DOMAIN_NAME + 'su/results.htm')

In [451]:
ol = driver.find_elements_by_tag_name('ol')[0]
li = ol.find_elements_by_tag_name('li')[5]
div = li.find_elements_by_tag_name('div')[0]
div = div.find_elements_by_class_name(RESULTS_CLASS_NAME)[0]

In [452]:
if div.text == 'Результаты':
    print(True)

True


In [453]:
wait = WebDriverWait(driver, 10)

In [454]:
li.click()

wait.until(EC.invisibility_of_element((By.CLASS_NAME, 'dropdown')))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="363e8bfd-c2a4-9445-980a-90e30738eeb3", element="fd5d4865-4d0b-db42-b9af-f1c70133235b")>

In [455]:
ol_in = li.find_elements_by_tag_name('ol')[0]
a = ol_in.find_elements_by_tag_name('a')[0]
assert a.text == 'Результаты'

In [456]:
a.click()

wait.until(EC.visibility_of_element_located((By.ID, 'search_words_field')))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="363e8bfd-c2a4-9445-980a-90e30738eeb3", element="490fb2d4-c074-704e-b2ee-57b7e322a93c")>

In [457]:
search_input_field = driver.find_element_by_id('search_words_field')

In [458]:
search_input_field.send_keys(SEARCH_STRING)

In [459]:
date_type_range = driver.find_element_by_id('date_type_RANGE')

In [460]:
date_type_range.click()

wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, 'input-field-component date-field-component')))

True

In [461]:
date_range_from = driver.find_element_by_id('date_range_from')
date_range_to = driver.find_element_by_id('date_range_to')

In [462]:
date_range_from.send_keys(Keys.COMMAND + 'A')
time.sleep(random.uniform(0, 4))
date_range_from.send_keys(Keys.BACKSPACE)
time.sleep(random.uniform(0, 4))
date_range_from.send_keys(DATE_START)
time.sleep(random.uniform(0, 4))

In [463]:
date_range_to.send_keys(Keys.COMMAND + 'A')
time.sleep(random.uniform(0, 4))
date_range_to.send_keys(Keys.BACKSPACE)
time.sleep(random.uniform(0, 4))
date_range_to.send_keys(DATE_END)
time.sleep(random.uniform(0, 4))

In [464]:
button_submit = driver.find_element_by_name('submit')

In [465]:
from selenium.common.exceptions import TimeoutException

In [466]:
try:
    button_submit.click()
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'result-sport')))
except TimeoutException:
    print(f'No data for requested date {DATE_START} :(')

In [467]:
div_results = driver.find_element_by_class_name('result-sport-list')

In [468]:
div_results_text = div_results.text

In [476]:
driver.close()

In [367]:
print(div_results_text[:150])

КИБЕРСПОРТ
FIFA. eSports Battle. Bundesliga. Круговой турнир
01 апр. 10:26 Бавария (Mr.Jab) - Байер 04 (Lomik)
0:0 (0:0, 0:0)
01 апр. 10:26 Боруссия Д


In [340]:
import uuid

In [341]:
uuid.uuid4()

UUID('1aec4e2c-51da-4b85-b38d-f1e809a51eb8')

In [480]:
def parse_date(line, date):
    try:
        ret_date = YEAR + ' ' + ' '.join(line.split(' ')[:3])
        ret_date = (datetime.strptime(ret_date, "%Y %d %b. %H:%M") + timedelta(seconds=240)).strftime('%Y-%m-%d %H:%M')
        return ret_date
    except:
        ret_date = date + ' ' + line.split(' ')[0]
        ret_date = (datetime.strptime(ret_date, "%d.%m.%Y %H:%M") + timedelta(seconds=240)).strftime('%Y-%m-%d %H:%M')
        return ret_date

In [483]:
parse_date('10:26 Бавария (Mr.Jab) - Байер 04 (Lomik)', '26.04.2020')

'2020-04-26 10:30'

# parse games

In [206]:
YEAR = '2020'

In [264]:
a = div_results_text.splitlines()[2]
date = YEAR + ' ' + ' '.join(a.split(' ')[:3])

datetime.strptime(date, "%Y %d %b. %H:%M").strftime('%Y-%m-%d %H:%M')

'2020-04-13 10:26'

In [218]:
a[14:]

'Бавария (d1pseN) - Байер 04 (Mr.Jab)'

In [219]:
re.findall(PLAYERS_PATTERN, a[14:])

['Бавария (d1pseN)', 'Байер 04 (Mr.Jab)']

In [301]:
from datetime import timedelta

In [470]:
games = []

cur_game_home = {}
cur_game_away = {}

TEAM_PLAYERS_PATTERN = r'[a-zA-Zа-яА-Я][0-9.a-zA-Zа-яА-Я\s]*\([0-9._а-яА-Яa-zA-Z]*\)'
RESULT_PATTERN = r'[0-9]*\:[0-9]*'

game_id = 0

for line in div_results_text.splitlines()[1:]:
    print(line)
    if 'FIFA. eSports Battle.' in line:
        league = line.split('.')[2].strip()
        continue
    if len(line.split()) > 3:
        date = YEAR + ' ' + ' '.join(line.split(' ')[:3])
        date = (datetime.strptime(date, "%Y %d %b. %H:%M") + timedelta(seconds=240)).strftime('%Y-%m-%d %H:%M')
        
        cur_game_home['game_id'] = game_id
        cur_game_away['game_id'] = game_id
        
        cur_game_home['date'] = date
        cur_game_away['date'] = date
        cur_game_home['league'] = league
        cur_game_away['league'] = league
        
        line_players = ' '.join(line.split(' ')[3:])
        
        teams = re.findall(TEAM_PLAYERS_PATTERN, line_players)
        assert len(teams) == 2
        
        team_home = teams[0].split('(')[0].strip()
        player_home = teams[0].split('(')[1].split(')')[0].strip()
        cur_game_home['team'] = team_home
        cur_game_home['player'] = player_home
        
        team_away = teams[1].split('(')[0].strip()
        player_away = teams[1].split('(')[1].split(')')[0].strip()
        cur_game_away['team'] = team_away
        cur_game_away['player'] = player_away
        
        continue
    elif line.lower() == 'матч отменен':
        cur_game_home, cur_game_away = {}, {}
        game_id += 1
        continue
    else:
        results = re.findall(RESULT_PATTERN, line)
        assert len(results) == 3
                
        ft_home = int(results[0].split(':')[0])
        ft_away = int(results[0].split(':')[1])
        
        cur_game_home['full_time'] = ft_home
        cur_game_away['full_time'] = ft_away
        
        fht_home = int(results[1].split(':')[0])
        fht_away = int(results[1].split(':')[1])
        cur_game_home['first_ht'] = fht_home
        cur_game_away['first_ht'] = fht_away
        
        sht_home = int(results[2].split(':')[0])
        sht_away = int(results[2].split(':')[1])
        cur_game_home['second_ht'] = sht_home
        cur_game_away['second_ht'] = sht_away
        
        cur_game_home['result_full_time'] = get_result(ft_home, ft_away)
        cur_game_away['result_full_time'] = get_result(ft_away, ft_home)
        cur_game_home['result_first_ht'] = get_result(fht_home, fht_away)
        cur_game_away['result_first_ht'] = get_result(fht_away, fht_home)
        cur_game_home['result_second_ht'] = get_result(sht_home, sht_away)
        cur_game_away['result_second_ht'] = get_result(sht_away, sht_home)

        games.append(cur_game_home)
        games.append(cur_game_away)
        print(cur_game_home)
        print(cur_game_away)
        cur_game_home, cur_game_away = {}, {}
        game_id += 1

FIFA. eSports Battle. Bundesliga. Круговой турнир
01 апр. 10:26 Бавария (Mr.Jab) - Байер 04 (Lomik)
0:0 (0:0, 0:0)
{'game_id': 0, 'date': '2020-04-01 10:30', 'league': 'Bundesliga', 'team': 'Бавария', 'player': 'Mr.Jab', 'full_time': 0, 'first_ht': 0, 'second_ht': 0, 'result_full_time': 'D', 'result_first_ht': 'D', 'result_second_ht': 'D'}
{'game_id': 0, 'date': '2020-04-01 10:30', 'league': 'Bundesliga', 'team': 'Байер 04', 'player': 'Lomik', 'full_time': 0, 'first_ht': 0, 'second_ht': 0, 'result_full_time': 'D', 'result_first_ht': 'D', 'result_second_ht': 'D'}
01 апр. 10:26 Боруссия Дортмунд (Skromnuy) - РБ Лейпциг (Arcos)
2:1 (0:1, 2:0)
{'game_id': 1, 'date': '2020-04-01 10:30', 'league': 'Bundesliga', 'team': 'Боруссия Дортмунд', 'player': 'Skromnuy', 'full_time': 2, 'first_ht': 0, 'second_ht': 2, 'result_full_time': 'W', 'result_first_ht': 'L', 'result_second_ht': 'W'}
{'game_id': 1, 'date': '2020-04-01 10:30', 'league': 'Bundesliga', 'team': 'РБ Лейпциг', 'player': 'Arcos', 'full

In [443]:
line

'05 апр. 12:38 Португалия (Arcos) - Испания Boulevard_Prospect)'

In [277]:
def get_result(score_1: int, score_2: int):
    if score_1 > score_2:
        return 'W'
    elif score_1 < score_2:
        return 'L'
    elif score_1 == score_2:
        return 'D'

In [395]:
df_games = pd.DataFrame(games)

In [396]:
df_games.player.unique()

array(['Mr.Jab', 'Kodak', 'KravaRK', 'Arcos', 'Skromnuy', 'dm1trena',
       'Foggy', 'Chellovekk', 'Olle', 'Inquisitor', 'Boulevard_Prospect',
       'orlovsky1', 'vrico', 'DangerDim77', 'nikkitta', 'MeLToSiK',
       'KRaftVK', 'labotryas', 'Upcake22', 'Kray', 'BlackStar98',
       'kalibra', 'FEARGGWP', 'Flewless_phoenix', 'PASHOK75', 'Калибра'],
      dtype=object)

In [314]:
df_games.to_csv('games_2020-04-12.csv', header=True, index=False)